In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import BandGapPredictor as bgp
import numpy as np
# from torchsummary import summary


import importlib

e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\partial_tensor.py:40: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)
e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\replicated_tensor.py:20: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)


In [2]:
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.set_float32_matmul_precision('high')
print(dev)

cuda


In [3]:

checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=10,
    verbose=False,
    mode='min'
)
# reload bgp 
importlib.reload(bgp)

# model = bgp.CNN.BandGap(lr=1e-3,model = bgp.CNN.CNNModelBig)

# model = model.load_from_checkpoint(checkpoint_path="checkpoints\epoch=0-val_loss=1.16-val_r2=-0.43.ckpt")
# m =  bgp.RESNET.ResNet(img_channels=3, num_layers=18, block=bgp.RESNET.BasicBlock, num_classes=1)
# m =  bgp.CNN.CNNModelBig()
m = bgp.RESNET.Resnet(3,1)
model = bgp.PLModule.BandGap(lr=1e-3,model =m).to(dev)
model.load_from_checkpoint(checkpoint_path="checkpoints\epoch=0-val_loss=0.94-val_r2=-69.20.ckpt")
trainer = pl.Trainer(accelerator='gpu', num_sanity_val_steps=0,devices=1, max_epochs=100,accumulate_grad_batches=16,log_every_n_steps=1,callbacks=[checkpoint_callback])

e:\miniconda3\envs\AI\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\miniconda3\envs\AI\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model Initialized


e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\utilities\parsing.py:269: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
# m(torch.rand(10,3,128,128,128).to(dev))

In [5]:
torch.cuda.empty_cache()

In [6]:
data  = bgp.dataloaders.LoadMolecules(molecules_root = r"T:\BandGap\3d-boxels-molecule-for-bandgap-prediction\Data",band_root =r"T:\BandGap\3d-boxels-molecule-for-bandgap-prediction" , max_samples=12500)
# data = [x for x in data]
print(len(data))
data_module = bgp.dataloaders.InMemoryDataModule(data,8,6)


12500


In [7]:
trainer.fit(model,data_module)

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory E:\Github\band-gap\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params
-------------------------------------
0 | loss    | RMSELoss | 0     
1 | r2score | R2Score  | 0     
2 | MAE     | L1Loss   | 0     
3 | model   | Resnet   | 33.5 M
-------------------------------------
33.5 M    Trainable params
0         Non-trainable params
33.5 M    Total params
134.029   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
import numpy as np
x , y = data_module.test_data[np.random.randint(0, len(data_module.test_data))]
x = torch.tensor(x).unsqueeze(0)
model.eval()
out = model(x)
print(out, y)
print(out-y)

In [ ]:
x , y = data_module.test_data[742]
# x to tensor
x = torch.tensor(x).unsqueeze(0).to(dev)
# model to eval
model.eval()
import time
start = time.time()
out = model(x)
end = time.time()
print(out, y)
print(end-start)

In [ ]:
e = trainer.test(model, data_module)

In [ ]:
e = trainer.validate(model, data_module)
